In [1]:
from botocore import UNSIGNED
from botocore.config import Config
import boto3
import os
import pyart
from datetime import datetime, timedelta
import os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import imageio

import matplotlib.cm as cm
import matplotlib.colors as mcolors

import osmnx as ox
from matplotlib.ticker import MultipleLocator

from math import cos, radians
import numpy as np
import numpy.ma as ma


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [12]:
# --- Configuration ---
# portland: 'KRTX'
# coast: 'KLGX'
# seattle: 'KATX'

station = 'KATX'
start_time = datetime(2024, 8, 18, 2, 0)
end_time = datetime(2024, 8, 18, 10, 0)
output_dir = './radar_files'
desired_fields = ['reflectivity', 'velocity', 'cross_correlation_ratio']

# --- Setup ---
os.makedirs(output_dir, exist_ok=True)
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = 'noaa-nexrad-level2'

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

def download_radar_files(station, start_time, end_time):
    files = []
    start_date = start_time.date()
    end_date = end_time.date()

    for single_date in daterange(start_date, end_date):
        prefix = f"{single_date:%Y/%m/%d}/{station}/"
        print(f"Checking prefix: {prefix}")

        response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
        if 'Contents' in response:
            for obj in response['Contents']:
                key = obj['Key']
                basename = os.path.basename(key)
                try:
                    # Parse time from filename, e.g. KRTX20230818_0104_V06
                    file_time = datetime.strptime(basename[4:17], "%Y%m%d_%H%M")
                    if start_time <= file_time <= end_time:
                        local_path = os.path.join(output_dir, basename)
                        if not os.path.exists(local_path):
                            print(f"Downloading {basename}")
                            s3.download_file(bucket, key, local_path)
                        files.append(local_path)
                except Exception as e:
                    # skip files with unexpected naming
                    continue
    return sorted(files)

# --- Run ---
files = download_radar_files(station, start_time, end_time)


Checking prefix: 2024/08/18/KATX/


In [2]:
def add_gridlines(ax, extent, spacing_lat=None, spacing_lon=None):
    lon_min, lon_max, lat_min, lat_max = extent
    if spacing_lat is None:
        spacing_lat = max((lat_max - lat_min) / 5, 0.01) 
    if spacing_lon is None:
        spacing_lon = max((lon_max - lon_min) / 5, 0.01)

    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    import matplotlib.ticker as mticker
    import numpy as np
    gl.xlocator = mticker.FixedLocator(np.arange(lon_min, lon_max + spacing_lon, spacing_lon))
    gl.ylocator = mticker.FixedLocator(np.arange(lat_min, lat_max + spacing_lat, spacing_lat))

    gl.xlabel_style = {'size': 8}
    gl.ylabel_style = {'size': 8}

def domain_size_km(lat_min, lat_max, lon_min, lon_max):
    """Calculate approximate max dimension of domain in km."""
    avg_lat = (lat_min + lat_max) / 2
    lat_km = (lat_max - lat_min) * 111
    lon_km = (lon_max - lon_min) * 111 * cos(radians(avg_lat))
    return max(lat_km, lon_km)

def download_osm_features(lat_min, lat_max, lon_min, lon_max):
    size_km = domain_size_km(lat_min, lat_max, lon_min, lon_max)
    print('Size: ', size_km, 'km')
    
    # Thresholds (example, tweak to your liking)
    # Smaller size -> more detail
    road_detail_levels = [
        (5, ["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "residential", "service", "track", "path"]),
        (20, ["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "residential"]),
        (50, ["motorway", "trunk", "primary", "secondary"]),
        (200, ["motorway", "trunk", "primary"]),
        (500, ["motorway", "trunk"]),
        (float('inf'), ["motorway"])
    ]

    city_detail_levels = [
        (50, ["city", "town", "village", "hamlet"]),
        (100, ["city", "town", "village"]),
        (150, ["city", "town"]),
        (500, ["city"])
    ]

    water_detail_levels = [
        (5, {"natural": ["water"], "waterway": ["river", "stream", "canal", "drain", "ditch"]}),
        (20, {"natural": ["water"], "waterway": ["river", "stream", "canal"]}),
        (500, {"natural": ["water"], "waterway": ["river"]}),
        (float('inf'), {"natural": ["water"]})
    ]

    def select_level(levels, size):
        for threshold, val in levels:
            if size <= threshold:
                return val
        return levels[-1][1]

    # Select tags based on domain size
    highway_tags = select_level(road_detail_levels, size_km)
    city_tags = select_level(city_detail_levels, size_km)
    water_tags = select_level(water_detail_levels, size_km)

    # Download roads
    print("Downloading roads with tags:", highway_tags)
    try:
        roads = ox.features.features_from_bbox(
            (lon_min, lat_min, lon_max, lat_max),
            tags={"highway": highway_tags}
        )
        roads = roads[roads.geometry.type.isin(["LineString", "MultiLineString"])]
    except Exception as e:
        print(f"⚠️ Failed to download roads: {e}")
        roads = None

    # Download water
    print("Downloading water features with tags:", water_tags)
    try:
        water = ox.features.features_from_bbox(
            (lon_min, lat_min, lon_max, lat_max),
            tags=water_tags
        )
        water = water[water.geometry.type.isin(["Polygon", "MultiPolygon", "LineString", "MultiLineString"])]
    except Exception as e:
        print(f"⚠️ Failed to download water features: {e}")
        water = None

    # Download cities
    print("Downloading cities with tags:", city_tags)
    try:
        cities = ox.features.features_from_bbox(
            (lon_min, lat_min, lon_max, lat_max),
            tags={"place": city_tags}
        )
        cities = cities[cities.geometry.type == "Point"]
    except Exception as e:
        print(f"⚠️ Failed to download cities: {e}")
        cities = None

    return roads, water, cities

    
def compute_osm_bbox(lat_min, lat_max, lon_min, lon_max, lat_rate, lon_rate, n_frames):
    # Compute full domain bounds after all shifts
    lat_max_final = lat_max + (n_frames - 1) * lat_rate
    lat_min_final = lat_min + (n_frames - 1) * lat_rate
    lon_max_final = lon_max + (n_frames - 1) * lon_rate
    lon_min_final = lon_min + (n_frames - 1) * lon_rate

    overall_lat_min = min(lat_min, lat_min_final)
    overall_lat_max = max(lat_max, lat_max_final)
    overall_lon_min = min(lon_min, lon_min_final)
    overall_lon_max = max(lon_max, lon_max_final)

    return overall_lat_min, overall_lat_max, overall_lon_min, overall_lon_max


def plot_field_sequence(
    data_dir: str,
    output_dir: str,
    field: str,
    start_time: datetime,
    end_time: datetime,
    lat_min: float = None,
    lat_max: float = None,
    lon_min: float = None,
    lon_max: float = None,
    lat_rate: float = 0.0,
    lon_rate: float = 0.0,
    dpi: int = 200,
    figsize: tuple = (6, 4),
    cmap: str = 'NWSRef',
    vmin: float = None,
    vmax: float = None,
    map_detail: str = '10m',
    features: bool = True,
    sweep: int = 0,
    bbox_inches: str = None,
    radar_code = 'KRTX'
):
    frames_dir = os.path.join(output_dir, 'frames')
    os.makedirs(frames_dir, exist_ok=True)

    all_files = sorted(glob.glob(os.path.join(data_dir, radar_code + '*_V06')))
    def file_time(fn):
        s = os.path.basename(fn)
        return datetime.strptime(s[4:17], '%Y%m%d_%H%M')
    files = [f for f in all_files if (start_time <= file_time(f) <= end_time)]
    files.sort(key=file_time)

    if not files:
        print(f"⚠️  No files found between {start_time} and {end_time} in {data_dir}")
        return

    if lat_min is None or lon_min is None:
        r0 = pyart.io.read(files[0])
        lats = r0.gate_latitude['data']
        lons = r0.gate_longitude['data']
        lat_min, lat_max = float(lats.min()), float(lats.max())
        lon_min, lon_max = float(lons.min()), float(lons.max())
        del r0

    overall_lat_min, overall_lat_max, overall_lon_min, overall_lon_max = compute_osm_bbox(
        lat_min, lat_max, lon_min, lon_max, lat_rate, lon_rate, len(files)
    )

    if features:
        road_gdf, water_gdf, city_gdf = download_osm_features(
            overall_lat_min, overall_lat_max, overall_lon_min, overall_lon_max
        )
    else:
        road_gdf, water_gdf, city_gdf = None, None, None

    domain_km = domain_size_km(overall_lat_min, overall_lat_max, overall_lon_min, overall_lon_max)

    frame_files = []
    for i, fn in enumerate(files):
        t = file_time(fn)
        print(t)
        c_lat_min = lat_min + i * lat_rate
        c_lat_max = lat_max + i * lat_rate
        c_lon_min = lon_min + i * lon_rate
        c_lon_max = lon_max + i * lon_rate
        extent = [c_lon_min, c_lon_max, c_lat_min, c_lat_max]

        try:
            radar = pyart.io.read(fn)
        except Exception as e:
            print(f"⚠️  Could not read {os.path.basename(fn)}: {e}")
            continue
        if field not in radar.fields:
            print(f"⚠️  {field} missing in {os.path.basename(fn)}, skipping")
            continue

        
        if field in radar.fields:
            field_data = radar.fields[field]['data']
            if isinstance(field_data, ma.MaskedArray):
                print(f"{field} frame {i}: min={field_data.min()}, max={field_data.max()}, masked={field_data.mask.sum()} / {field_data.size}")
            else:
                print(f"{field} frame {i}: min={np.min(field_data)}, max={np.max(field_data)}")

        fig = plt.figure(figsize=figsize, dpi=dpi)
        ax = plt.subplot(projection=ccrs.PlateCarree())
        ax.set_extent(extent, crs=ccrs.PlateCarree())

        # Add map features
        ax.add_feature(cfeature.COASTLINE.with_scale(map_detail))
        ax.add_feature(cfeature.BORDERS.with_scale(map_detail))
        ax.add_feature(cfeature.STATES.with_scale(map_detail))

        # Plot roads
        if road_gdf is not None:
            road_gdf.plot(ax=ax, color='dimgray', linewidth=0.4, transform=ccrs.PlateCarree(), zorder=3)

        # Plot water
        if water_gdf is not None:
            water_gdf.plot(ax=ax, color='dodgerblue', linewidth=0.6, alpha=0.6, transform=ccrs.PlateCarree(), zorder=2)

        if city_gdf is not None:
            # Marker size scales with zoom: larger markers for small domains
            city_size = 10 if domain_km <= 30 else 4
            city_gdf.plot(ax=ax, color='black', markersize=city_size, transform=ccrs.PlateCarree(), zorder=4)

            # Filter cities within current frame extent
            lon_min_cities, lon_max_cities, lat_min_cities, lat_max_cities = extent
            visible_cities = city_gdf.cx[lon_min_cities:lon_max_cities, lat_min_cities:lat_max_cities]

            # Always label cities within current extent
            for _, row in visible_cities.iterrows():
                name = row.get('name')
                if name:
                    ax.text(
                        row.geometry.x,
                        row.geometry.y,
                        name,
                        transform=ccrs.PlateCarree(),
                        fontsize=7 if domain_km <= 30 else 5,
                        ha='left',
                        va='bottom',
                        color='black',
                        zorder=5
                    )

        # Plot radar data
        disp = pyart.graph.RadarMapDisplay(radar)

        field_title_dict = {
            'reflectivity': 'Reflectivity',
            'velocity': 'Radial Velocity',
            'cross_correlation_ratio': 'Correlation Coefficient'
        }

        field_label_dict = {
            'reflectivity': 'Reflectivity (dBZ)',
            'velocity': 'Radial Velocity (m/s)',
            'cross_correlation_ratio': 'Correlation Coefficient'
        }

        title_text = field_title_dict.get(field)
        colorbar_label = field_label_dict.get(field)

        # Plot radar data but suppress Py-ART's default colorbar
        mesh = disp.plot_ppi_map(
            field, sweep,
            ax=ax,
            vmin=vmin, vmax=vmax, cmap=cmap,
            lat_lines=None, lon_lines=None,
            projection=ccrs.PlateCarree(),
            min_lat=c_lat_min, max_lat=c_lat_max,
            min_lon=c_lon_min, max_lon=c_lon_max,
            resolution=map_detail,
            colorbar_flag=False
        )

        # Add our custom colorbar
        norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
        mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
        mappable.set_array([])  # Required to make the colorbar work

        cb = plt.colorbar(mappable, ax=ax, shrink=0.7, pad=0.02)
        cb.set_label(colorbar_label, fontsize=8)

        # Add gridlines
        add_gridlines(ax, extent)

        plt.title(f"{title_text} at {t:%Y-%m-%d %H:%M} UTC", fontsize=10)
        out_png = os.path.join(frames_dir, f"frame_{i:03d}.png")
        plt.savefig(out_png, bbox_inches=bbox_inches, pad_inches=0)
        plt.close(fig)

        frame_files.append(out_png)

    if not frame_files:
        print(f"⚠️  All files in the window were missing field `{field}`. No frames to stitch.")
        return

    imgs = [imageio.imread(fn) for fn in frame_files]
    gif_path = os.path.join(output_dir, f"{field}_{start_time:%H%M}_{end_time:%H%M}.gif")
    imageio.mimsave(gif_path, imgs, fps=4)
    print(f"✅  Saved {len(frame_files)} frames + GIF → {gif_path}")


In [3]:
def extract_field_extrema(
    data_dir: str,
    output_txt: str,
    field: str,
    start_time: datetime,
    end_time: datetime,
    lat_min: float,
    lat_max: float,
    lon_min: float,
    lon_max: float,
    lat_rate: float = 0.0,
    lon_rate: float = 0.0,
    radar_code = 'KRTX'
):
    def file_time(fn):
        s = os.path.basename(fn)
        return datetime.strptime(s[4:17], '%Y%m%d_%H%M')

    all_files = sorted(glob.glob(os.path.join(data_dir, radar_code + '*_V06')))
    files = [f for f in all_files if (start_time <= file_time(f) <= end_time)]
    files.sort(key=file_time)

    if not files:
        print(f"⚠️ No files found between {start_time} and {end_time} in {data_dir}")
        return

    with open(output_txt, 'w') as out:
        header = "time, min_val, min_lat, min_lon, max_val, max_lat, max_lon\n"
        out.write(header)

        for i, fn in enumerate(files):
            t = file_time(fn)

            # Compute moving box for this frame
            c_lat_min = lat_min + i * lat_rate
            c_lat_max = lat_max + i * lat_rate
            c_lon_min = lon_min + i * lon_rate
            c_lon_max = lon_max + i * lon_rate

            try:
                radar = pyart.io.read(fn)
            except Exception as e:
                print(f"⚠️ Could not read {os.path.basename(fn)}: {e}")
                continue

            if field not in radar.fields:
                print(f"⚠️ {field} missing in {os.path.basename(fn)}, skipping")
                continue

            field_data = radar.fields[field]['data']
            lats = radar.gate_latitude['data']
            lons = radar.gate_longitude['data']

            # Create mask for gates within the moving lat/lon box
            in_box = (
                (lats >= c_lat_min) & (lats <= c_lat_max) &
                (lons >= c_lon_min) & (lons <= c_lon_max)
            )

            if isinstance(field_data, ma.MaskedArray):
                valid = in_box & ~field_data.mask
            else:
                valid = in_box

            if not np.any(valid):
                print(f"⚠️ No valid data in box for {t:%Y-%m-%d %H:%M}, skipping")
                continue

            data = field_data[valid]
            lat_valid = lats[valid]
            lon_valid = lons[valid]

            min_idx = np.argmin(data)
            max_idx = np.argmax(data)

            min_val = data[min_idx]
            max_val = data[max_idx]
            min_lat, min_lon = lat_valid[min_idx], lon_valid[min_idx]
            max_lat, max_lon = lat_valid[max_idx], lon_valid[max_idx]

            out.write(f"{t:%Y-%m-%d %H:%M}, {min_val:.2f}, {min_lat:.4f}, {min_lon:.4f}, {max_val:.2f}, {max_lat:.4f}, {max_lon:.4f}\n")
            print(f"{t:%Y-%m-%d %H:%M} — min={min_val:.2f} @ ({min_lat:.2f}, {min_lon:.2f}), max={max_val:.2f} @ ({max_lat:.2f}, {max_lon:.2f})")

    print(f"✅ Saved extrema data to {output_txt}")


In [ ]:
# tornado domain
for field, cmap, vmin, vmax, folder, sweep in zip(['reflectivity', 'velocity', 'cross_correlation_ratio'], ['pyart_NWSRef', 'pyart_NWSVel', 'pyart_Carbone42'], [-20, -30, 0], [75, 30, 1], ['./radar_animations/tor/reflectivity', './radar_animations/tor/velocity', './radar_animations/tor/correlation'], [0, 1, 0]):
    plot_field_sequence(
        data_dir   = './radar_files',
        output_dir = folder,
        field      = field,
        start_time = datetime(2024,8,17,22,00),
        end_time   = datetime(2024,8,17,23,20),
        lat_min    = 44.5,
        lat_max    = 45,
        lon_min    = -122.5,
        lon_max    = -121.75,
        lon_rate   = 0,
        cmap       = cmap,
        vmin       = vmin,
        vmax       = vmax,
        features = True,
        figsize = (6, 6),
        sweep      = sweep,
        bbox_inches= 'tight'
    )

    
    extract_field_extrema(
        data_dir = './radar_files',
        output_txt=folder + '/extrema.txt',
        field=field,
        start_time = datetime(2024,8,17,22,00),
        end_time   = datetime(2024,8,17,23,20),
        lat_min    = 44.5,
        lat_max    = 45,
        lon_min    = -122.5,
        lon_max    = -121.75,
    )

Size:  59.122932520638514 km
2024-08-17 22:03:00
reflectivity frame 0: min=-32.0, max=68.0, masked=14095223 / 16488000
2024-08-17 22:08:00
reflectivity frame 1: min=-32.0, max=68.0, masked=14069216 / 16488000
2024-08-17 22:13:00
reflectivity frame 2: min=-32.0, max=66.5, masked=14038425 / 16488000
2024-08-17 22:18:00
reflectivity frame 3: min=-32.0, max=67.0, masked=14013757 / 16488000
2024-08-17 22:23:00
reflectivity frame 4: min=-32.0, max=68.5, masked=13983718 / 16488000
2024-08-17 22:28:00
reflectivity frame 5: min=-32.0, max=70.5, masked=13957915 / 16488000
2024-08-17 22:33:00
reflectivity frame 6: min=-31.5, max=69.5, masked=13939693 / 16488000
2024-08-17 22:38:00
reflectivity frame 7: min=-32.0, max=72.0, masked=13279773 / 15828480
2024-08-17 22:43:00
reflectivity frame 8: min=-31.5, max=70.5, masked=13889015 / 16488000
2024-08-17 22:47:00
reflectivity frame 9: min=-32.0, max=71.5, masked=14502621 / 17147520
2024-08-17 22:52:00
reflectivity frame 10: min=-31.5, max=72.0, masked=

C:\Users\miles\AppData\Local\Temp\ipykernel_35920\170501549.py:301: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imageio.imread(fn) for fn in frame_files]


✅  Saved 16 frames + GIF → ./radar_animations/tor/reflectivity\reflectivity_2200_2320.gif
2024-08-17 22:03 — min=-5.00 @ (44.98, -122.44), max=59.50 @ (44.50, -122.05)
2024-08-17 22:08 — min=-5.00 @ (45.00, -122.49), max=61.50 @ (44.55, -122.07)
2024-08-17 22:13 — min=-5.00 @ (44.97, -122.42), max=61.00 @ (44.69, -122.18)
2024-08-17 22:18 — min=-4.50 @ (44.99, -122.48), max=62.00 @ (44.58, -122.10)
2024-08-17 22:23 — min=-4.50 @ (44.99, -122.39), max=65.00 @ (44.67, -122.03)
2024-08-17 22:28 — min=-4.50 @ (44.99, -122.48), max=70.50 @ (44.67, -122.03)
2024-08-17 22:33 — min=-5.00 @ (44.99, -122.46), max=69.50 @ (44.69, -122.02)
2024-08-17 22:38 — min=-4.50 @ (44.99, -122.49), max=72.00 @ (44.74, -122.04)
2024-08-17 22:43 — min=-4.00 @ (45.00, -122.49), max=70.50 @ (44.79, -122.07)
2024-08-17 22:47 — min=-4.00 @ (44.99, -122.43), max=71.50 @ (44.81, -122.08)
2024-08-17 22:52 — min=-4.50 @ (44.99, -122.45), max=72.00 @ (44.90, -122.07)
2024-08-17 22:57 — min=-3.00 @ (45.00, -121.91), max

C:\Users\miles\AppData\Local\Temp\ipykernel_35920\170501549.py:301: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imageio.imread(fn) for fn in frame_files]


✅  Saved 16 frames + GIF → ./radar_animations/tor/velocity\velocity_2200_2320.gif
2024-08-17 22:03 — min=-24.00 @ (44.51, -122.26), max=24.00 @ (44.70, -121.98)
2024-08-17 22:08 — min=-24.00 @ (44.64, -121.80), max=24.00 @ (44.66, -121.82)
2024-08-17 22:13 — min=-24.00 @ (44.53, -121.77), max=24.00 @ (44.52, -121.79)
2024-08-17 22:18 — min=-24.00 @ (44.78, -121.88), max=24.00 @ (44.83, -121.95)
2024-08-17 22:23 — min=-24.00 @ (44.56, -121.83), max=24.00 @ (44.57, -121.83)
2024-08-17 22:28 — min=-24.00 @ (44.63, -121.85), max=24.00 @ (44.63, -121.85)
2024-08-17 22:33 — min=-24.00 @ (44.69, -121.87), max=24.00 @ (44.68, -121.84)
2024-08-17 22:38 — min=-24.00 @ (44.72, -121.85), max=24.00 @ (44.65, -121.87)
2024-08-17 22:43 — min=-26.50 @ (44.91, -121.95), max=26.50 @ (44.69, -121.88)
2024-08-17 22:47 — min=-28.50 @ (44.74, -121.86), max=28.50 @ (44.54, -122.08)
2024-08-17 22:52 — min=-28.50 @ (44.97, -121.98), max=28.50 @ (44.96, -121.98)
2024-08-17 22:57 — min=-26.50 @ (44.57, -121.83),

C:\Users\miles\AppData\Local\Temp\ipykernel_35920\170501549.py:301: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imageio.imread(fn) for fn in frame_files]


✅  Saved 16 frames + GIF → ./radar_animations/tor/correlation\cross_correlation_ratio_2200_2320.gif
2024-08-17 22:03 — min=0.21 @ (44.95, -121.77), max=1.05 @ (44.96, -121.77)
2024-08-17 22:08 — min=0.21 @ (44.97, -121.76), max=1.05 @ (44.97, -121.76)
2024-08-17 22:13 — min=0.21 @ (44.99, -122.17), max=1.05 @ (44.96, -121.76)
2024-08-17 22:18 — min=0.21 @ (44.98, -122.32), max=1.05 @ (44.97, -121.75)
2024-08-17 22:23 — min=0.21 @ (44.96, -121.76), max=1.05 @ (44.95, -121.76)
2024-08-17 22:28 — min=0.21 @ (44.94, -121.81), max=1.05 @ (44.95, -121.77)
2024-08-17 22:33 — min=0.21 @ (44.87, -121.98), max=1.05 @ (44.97, -121.75)
2024-08-17 22:38 — min=0.21 @ (44.68, -121.84), max=1.05 @ (44.97, -121.75)
2024-08-17 22:43 — min=0.21 @ (44.79, -121.83), max=1.05 @ (44.95, -121.75)
2024-08-17 22:47 — min=0.21 @ (44.89, -121.94), max=1.05 @ (44.94, -121.76)
2024-08-17 22:52 — min=0.21 @ (44.93, -121.93), max=1.05 @ (44.97, -121.75)
2024-08-17 22:57 — min=0.21 @ (44.92, -121.82), max=1.05 @ (44.9

In [5]:
# following domain
for field, cmap, vmin, vmax, folder, sweep in zip(['reflectivity', 'velocity', 'cross_correlation_ratio'], ['pyart_NWSRef', 'pyart_NWSVel', 'pyart_Carbone42'], [-20, -30, 0], [75, 30, 1], ['./radar_animations/follow/reflectivity', './radar_animations/follow/velocity', './radar_animations/follow/correlation'], [0, 1, 0]):
    '''if field != 'reflectivity':
        plot_field_sequence(
            data_dir   = './radar_files',
            output_dir = folder,
            field      = field,
            start_time = datetime(2024,8,17,21,00),
            end_time   = datetime(2024,8,18,4,00),
            lat_min    = 43.5,
            lat_max    = 44.5,
            lon_min    = -123.5,
            lon_max    = -121.5,
            lat_rate   = .0425,
            lon_rate   = .0072,
            cmap       = cmap,
            vmin       = vmin,
            vmax       = vmax,
            features   = True,
            sweep      = sweep
        )'''

    extract_field_extrema(
        data_dir = './radar_files',
        output_txt=folder + '/extrema.txt',
        field=field,
        start_time = datetime(2024,8,17,21,00),
        end_time   = datetime(2024,8,18,4,00),
        lat_min    = 43.5,
        lat_max    = 44.5,
        lon_min    = -123.5,
        lon_max    = -121.5,
        lat_rate   = .0425,
        lon_rate   = .0072,
    )
    

2024-08-17 21:01 — min=-0.50 @ (44.50, -122.62), max=53.00 @ (43.56, -122.42)
2024-08-17 21:05 — min=-1.00 @ (44.52, -122.60), max=56.00 @ (43.58, -122.46)
2024-08-17 21:10 — min=-1.00 @ (44.58, -122.52), max=55.00 @ (43.78, -122.28)
2024-08-17 21:14 — min=-2.00 @ (44.63, -122.66), max=55.00 @ (43.86, -122.29)
2024-08-17 21:18 — min=-2.00 @ (44.66, -122.70), max=56.00 @ (43.83, -122.30)
2024-08-17 21:22 — min=-2.50 @ (44.65, -122.57), max=55.00 @ (44.43, -122.86)
2024-08-17 21:26 — min=-2.50 @ (44.71, -123.02), max=55.50 @ (44.47, -122.88)
2024-08-17 21:30 — min=-3.00 @ (44.69, -122.81), max=56.50 @ (43.97, -122.31)
2024-08-17 21:35 — min=-4.00 @ (44.83, -122.73), max=59.00 @ (44.39, -122.64)
2024-08-17 21:39 — min=-4.00 @ (44.85, -122.76), max=59.50 @ (44.38, -122.00)
2024-08-17 21:44 — min=-5.00 @ (44.92, -122.70), max=62.00 @ (44.02, -122.37)
2024-08-17 21:49 — min=-6.00 @ (44.97, -122.91), max=60.50 @ (44.42, -122.07)
2024-08-17 21:53 — min=-6.50 @ (45.00, -122.90), max=60.50 @ (44

In [ ]:
# MCS from seattle domain
for field, cmap, vmin, vmax, folder, sweep in zip(['reflectivity', 'velocity', 'cross_correlation_ratio'], ['pyart_NWSRef', 'pyart_NWSVel', 'pyart_Carbone42'], [-20, -30, 0], [75, 30, 1], ['./radar_animations/katx/reflectivity', './radar_animations/katx/velocity', './radar_animations/katx/correlation'], [0, 1, 0]):
    plot_field_sequence(
        data_dir   = './radar_files',
        output_dir = folder,
        field      = field,
        start_time = datetime(2024,8,18,2,00),
        end_time   = datetime(2024,8,18,10,00),
        lat_min    = 45.5,
        lat_max    = 46.5,
        lon_min    = -123.5,
        lon_max    = -121,
        lat_rate   = .042,
        lon_rate   = 0,
        cmap       = cmap,
        vmin       = vmin,
        vmax       = vmax,
        features   = True,
        sweep      = sweep,
        radar_code = 'KATX'
    )

    extract_field_extrema(
        data_dir = './radar_files',
        output_txt=folder + '/extrema.txt',
        field=field,
        start_time = datetime(2024,8,18,2,00),
        end_time   = datetime(2024,8,18,10,00),
        lat_min    = 45.5,
        lat_max    = 46.5,
        lon_min    = -123.5,
        lon_max    = -121,
        lat_rate   = .042,
        lon_rate   = 0,
        radar_code = 'KATX'
    )
    

In [ ]:
# MCS from coast (langley hill) domain
for field, cmap, vmin, vmax, folder, sweep in zip(['reflectivity', 'velocity', 'cross_correlation_ratio'], ['pyart_NWSRef', 'pyart_NWSVel', 'pyart_Carbone42'], [-20, -30, 0], [75, 30, 1], ['./radar_animations/klgx/reflectivity', './radar_animations/klgx/velocity', './radar_animations/klgx/correlation'], [0, 1, 0]):
    plot_field_sequence(
        data_dir   = './radar_files',
        output_dir = folder,
        field      = field,
        start_time = datetime(2024,8,18,1,00),
        end_time   = datetime(2024,8,18,7,00),
        lat_min    = 45,
        lat_max    = 47,
        lon_min    = -125,
        lon_max    = -121.5,
        lat_rate   = .028,
        lon_rate   = -.014,
        cmap       = cmap,
        vmin       = vmin,
        vmax       = vmax,
        features   = True,
        sweep      = sweep,
        radar_code = 'KLGX'
    )

    extract_field_extrema(
        data_dir = './radar_files',
        output_txt=folder + '/extrema.txt',
        field=field,
        start_time = datetime(2024,8,18,1,00),
        end_time   = datetime(2024,8,18,7,00),
        lat_min    = 45,
        lat_max    = 47,
        lon_min    = -125,
        lon_max    = -121.5,
        lat_rate   = .028,
        lon_rate   = -.014,
        radar_code = 'KLGX'
    )